In [1]:
!pip install pubchempy

  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=3d05cf1e791695fc8211fc079e270bb85937d60199758b3b08d7534a23330772
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [2]:
import re
import itertools
import json
import sys
import os
import time
import traceback
from lxml import etree
import pubchempy as pcp
import xml.etree.ElementTree as ET

In [3]:
database = open("/kaggle/input/drugbank-data/full database.xml", 'r')
tree = ET.parse(database)
root = tree.getroot()

In [7]:
k = 0
f = open("alldrugs.txt", 'w')
Data = []
sdata = []
name = None
for drug in root:
    k = k +1

    id_ = drug.find("{http://www.drugbank.ca}drugbank-id")
    if id_ is not None:
        d_id_ = id_.text
        line = d_id_

    name = drug.find("{http://www.drugbank.ca}name")
    if name is not None:
        d_name = name.text  
        line += '|' + name.text

    state = drug.find("{http://www.drugbank.ca}state")
    if state is not None:
        d_state = state.text

    description = drug.find("{http://www.drugbank.ca}description")
    if description is not None:
        d_description = description.text

    indication =  drug.find("{http://www.drugbank.ca}indication")
    if indication is not None:
        d_indication = indication.text

    #----------synonyms ----------
    synonyms = drug.find("{http://www.drugbank.ca}synonyms")
    S = []
    for t in synonyms:
        S.append(t.text)
        line = line + "|" + t.text

    #----------products ----------
    products = drug.find("{http://www.drugbank.ca}products")
    P = []
    for t in products:
        P.append(t.find("{http://www.drugbank.ca}name").text)
        line = line + "|" + t.find("{http://www.drugbank.ca}name").text

    #----------international-brands ----------
    international_brands = drug.find("{http://www.drugbank.ca}international-brands")
    B = []
    for t in international_brands:
        B.append(t.find("{http://www.drugbank.ca}name").text)
        line = line + "|" + t.find("{http://www.drugbank.ca}name").text
        
    #----------formula ----------
    F = []
    exp_properties = drug.find("{http://www.drugbank.ca}experimental-properties")
    calc_properties = drug.find("{http://www.drugbank.ca}calculated-properties")
    if calc_properties is not None:
        for p in calc_properties:
            if p.find("{http://www.drugbank.ca}kind").text == 'Molecular Formula':
                F.append(p.find("{http://www.drugbank.ca}value").text)
    for p in exp_properties:
        if p.find("{http://www.drugbank.ca}kind").text == 'Molecular Formula':
            F.append(p.find("{http://www.drugbank.ca}value").text)
            
    sdata.append({"id":d_id_,\
                 "name":d_name,\
                 "synonyms":S,
                 "products":P,
                 "international-brands":B})

    Data.append({"id":d_id_, \
                "name":d_name,\
                "description":d_description,\
                "state": d_state,\
                "indication": d_indication,\
                "synonyms":S, \
                "formula":F})
    f.write(line)
    f.write("\n")

f = open('/kaggle/working/alldrugs.txt')

drug_id_list = f.read().split('\n')
drug_dct = dict()

for drug in drug_id_list:
    lst = drug.split('|')
    drug_dct[lst[0]] = list(set([x.lower() for x in lst[1:]]))

In [81]:
def find_drugbank(drug_name, drug_dct):
    drug_name = drug_name.lower()
    for k, v in drug_dct.items():
        if drug_name in v:
            return k
    for k, v in drug_dct.items():
        for _v in v:
            if _v.find(drug_name) != -1 or drug_name.find(_v) != -1:
                if len(_v) > 3:
                    return k
    return None

def get_smiles_from_pubchem_compounds(drug_name):
    results = pcp.get_compounds(drug_name, 'name')
    for compound in results:
        return compound.isomeric_smiles
    return None

def get_smiles_from_pubchem_substances(drug_name):
    results = pcp.get_substances(drug_name, 'name')
    if results == []:
        print(f"{drug_name} not found")
        return None
    try:
        compound = results[0].standardized_compound
    except:
        print(f"{drug_name} compound error")
        return None
    if compound == None:
        print(f"{drug_name} does not have standardized compound")
        return None
    return compound.isomeric_smiles

def find_smiles(drug):
    smiles = get_smiles_from_pubchem_compounds(drug)
    if smiles == None:
        smiles = get_smiles_from_pubchem_substances(drug)
    return smiles

In [87]:
from tqdm import tqdm
import time

def find_by_name(drug_all, drug_name_dct):
    drug_dct = dict()
    for drug_name in tqdm(drug_all):
        drugbank_id = find_drugbank(drug_name, drug_name_dct)
        if drugbank_id != None:
            try:
                smiles = find_smiles(drugbank_id)
            except:
                time.sleep(1)
                smiles = find_smiles(drugbank_id)
        else:
            try:
                smiles = find_smiles(drug_name)
            except:
                time.sleep(1)
                smiles = find_smiles(drug_name)
        drug_dct[drug_name] = {"id": str(drugbank_id), 
                               "smiles": str(smiles)}

    return drug_dct

def find_by_id(drug_all):
    drug_dct = dict()
    for drugbank_id in tqdm(drug_all):
        try:
            smiles = find_smiles(drugbank_id)
        except:
            time.sleep(1)
            smiles = find_smiles(drugbank_id)
        drug_dct[drugbank_id] = {"id": str(drugbank_id), 
                                 "smiles": str(smiles)}
    return drug_dct

def add_formula(smiles, data):
    for k, v in smiles.items():
        for i in range(len(data)):
            if data[i]['id'] == v['id'] and len(data[i]['formula']) > 0:
                smiles[k]['formula'] = str(data[i]['formula'][0])
        if 'formula' not in v.keys():
            smiles[k]['formula'] = 'None'
    return smiles

def merge(smiles, smiles_id):
    for k, v in smiles.items():
        if v['smiles'] == 'None':
            for k1, v1 in smiles_id.items():
                if v['id'] == v1['id']:
                    smiles[k]['smiles'] = k1[v['id']]['smiles']
    return smiles

In [124]:
f = open('/kaggle/input/ddi-entities/all_drug_n.txt')
ddi_brand = [x.lower() for x in f.read().split('\n')[:-1]]
smiles = find_by_name(ddi_brand, drug_dct)
smiles_id = find_by_id(ddi_brand)
smiles = merge(smiles, smiles_id)
smiles = add_formula(smiles, Data)

  1%|▏         | 3/223 [00:01<02:32,  1.44it/s]

(-)-erythro-fluorocitrate not found


  2%|▏         | 4/223 [00:02<02:37,  1.39it/s]

(2-adamantyl)hexamethylenimine not found


  7%|▋         | 15/223 [00:07<01:50,  1.89it/s]

...

 92%|█████████▏| 206/223 [02:07<00:08,  2.00it/s]

reseveratrol not found


 97%|█████████▋| 216/223 [02:13<00:03,  1.82it/s]

toxin a compound error


100%|██████████| 223/223 [02:15<00:00,  1.64it/s]


In [125]:
name, iden, smi, formula = [], [], [], []
for k, v in smiles.items():
    name.append(k)
    iden.append(v['id'])
    smi.append(v['smiles'])
    formula.append(v['formula'])

In [126]:
import pandas as pd

full = pd.DataFrame({"name": name,
                     "id": iden,
                     "smiles": smi,
                     "formula": formula})

In [127]:
full

,name,id,smiles,formula
0,(+)-nanm,None,C[C@@H]1[C@@H]2CC3=C([C@]1(CCN2CC=C)C)C=C(C=C3)O,None
1,(-)-nanm,None,C[C@H]1[C@H]2CC3=C([C@@]1(CCN2CC=C)C)C=C(C=C3)O,None
2,(-)-erythro-fluorocitrate,None,None,None
3,(2-adamantyl)hexamethylenimine,None,None,None
4,"1,3-difluoro-2-propanol",DB00742,C([C@H]([C@H]([C@@H]([C@@H](CO)O)O)O)O)O,C6H14O6
...,...,...,...,...
202,trimethyl lead,DB09089,CCC(COC(=O)C1=CC(=C(C(=C1)OC)OC)OC)(C2=CC=CC=C...,C22H29NO5
203,tuftsin,None,C[C@H]([C@@H](C(=O)N[C@@H](CCCCN)C(=O)N1CCC[C@...,None
204,tyramine,DB08841,C1=CC(=CC=C1CCN)O,C8H11NO
205,vanillin,None,COC1=C(C=CC(=C1)C=O)O,None


In [129]:
full.to_csv('drug_n_full.csv')

In [138]:
d1 = pd.read_csv('/kaggle/working/brand_full.csv')
d2 = pd.read_csv('/kaggle/working/drug_full.csv')
d3 = pd.read_csv('/kaggle/working/drug_n_full.csv')
d4 = pd.read_csv('/kaggle/working/group_full.csv')

In [146]:
full_df = pd.concat([d1, d2, d3, d4], ignore_index=True)

In [155]:
full_df.to_csv('full.csv', index=False)

In [ ]:
# Drug_n: Not found 63/223
# Drug: Not found 120/1623
# Brand: Not found 9/464
# Group: Not found 907/999